In [1]:
import pandas as pd
import nltk
import pprint
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import sent_tokenize
from nltk.stem import PorterStemmer
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

In [2]:
#dataset https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis
#financial-sentiment-analysis
df = pd.read_csv('data.csv')

In [3]:
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [4]:
df.tail()

,Sentence,Sentiment
5837,RISING costs have forced packaging producer Hu...,negative
5838,Nordic Walking was first used as a summer trai...,neutral
5839,"According shipping company Viking Line , the E...",neutral
5840,"In the building and home improvement trade , s...",neutral
5841,HELSINKI AFX - KCI Konecranes said it has won ...,positive


In [5]:
#Pre-Processing
text = df['Sentence']
filtered_words = []
sen = []
words = []
final=''
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
for i in text:
    word_token = word_tokenize(i)                 #tokenize
    filtered_words = []
    for w in word_token:
        w = w.lower()                            #lower case
        if w not in stop_words:
            if w not in punc:
                for j in w:
                    if j in punc:
                        w=w.replace(j,'')          #remove punctuation
                filtered_words.append(ps.stem(w))  #Stemming
    words.append(filtered_words)
df.insert(2,'Final',words)

In [6]:
texts = df['Final']
final = []
for i in texts:
    final.append(' '.join(map(str, i)))

In [7]:
print(final[0])

geosolut technolog leverag benefon s gp solut provid locat base search technolog commun platform locat relev multimedia content new power commerci model


In [8]:
df['Final'] = final
df.head()

,Sentence,Sentiment,Final
0,The GeoSolutions technology will leverage Bene...,positive,geosolut technolog leverag benefon s gp solut ...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi low 150 250 bk real possibl
2,"For the last quarter of 2010 , Componenta 's n...",positive,last quarter 2010 componenta s net sale doubl ...
3,According to the Finnish-Russian Chamber of Co...,neutral,accord finnishrussian chamber commerc major co...
4,The Swedish buyout firm has sold its remaining...,neutral,swedish buyout firm sold remain 224 percent st...


In [9]:
print(type(df['Final'][0]))
df['Final'] = df['Final'].str.replace('[0-9]','')
print(df['Final'][0])

<class 'str'>
geosolut technolog leverag benefon s gp solut provid locat base search technolog commun platform locat relev multimedia content new power commerci model


<ipython-input-9-ffd7dc89f02f>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Final'] = df['Final'].str.replace('[0-9]','')


In [10]:
#inverted index
def generate_inverted_index(data: list):
    inv_idx_dict = {}
     
    for index, doc_text in enumerate(data):
        for word in doc_text.split():
            if word not in inv_idx_dict.keys():
                inv_idx_dict[word] = [index]
            elif index not in inv_idx_dict[word]:
                inv_idx_dict[word].append(index)
    return inv_idx_dict
inverted_index = generate_inverted_index(df['Final'])
print(len(inverted_index))

8282


In [11]:
inverted_index

{'geosolut': [0, 412],
 'technolog': [0,
  59,
  109,
  137,
  300,
  412,
  427,
  436,
  473,
  558,
  657,
  672,
  679,
  682,
  712,
  720,
  757,
  758,
  823,
  849,
  941,
  990,
  1018,
  1105,
  1124,
  1215,
  1329,
  1462,
  1499,
  1516,
  1597,
  1600,
  1629,
  1638,
  1689,
  1719,
  1724,
  1879,
  1925,
  1941,
  1970,
  2072,
  2129,
  2166,
  2176,
  2216,
  2263,
  2357,
  2484,
  2487,
  2534,
  2675,
  2696,
  2752,
  2828,
  2936,
  3031,
  3056,
  3067,
  3079,
  3085,
  3181,
  3237,
  3238,
  3242,
  3248,
  3272,
  3308,
  3353,
  3484,
  3487,
  3510,
  3564,
  3685,
  3993,
  4048,
  4082,
  4093,
  4110,
  4116,
  4140,
  4145,
  4149,
  4158,
  4256,
  4341,
  4439,
  4449,
  4481,
  4497,
  4515,
  4578,
  4615,
  4815,
  4823,
  4864,
  4869,
  5084,
  5155,
  5162,
  5167,
  5191,
  5271,
  5288,
  5329,
  5342,
  5389,
  5410,
  5496,
  5497,
  5534,
  5566,
  5604,
  5709,
  5780,
  5816],
 'leverag': [0, 858, 3958],
 'benefon': [0, 300, 1053, 1312,

In [12]:
#posting list
final_filtered_Sentence = df['Final']
vocab = []
postings = {}
def generate_positional_index(data: list):
  for index,doc_text in enumerate(data):
      for word in doc_text.split():
          if word not in vocab:
              vocab.append(word)
          wordId = vocab.index(word)
          if word not in postings:
              postings[word] = [index]
          else:
              postings[word].append(index)
          #print(wordId,word)
  #print(postings)
  for i in postings:
      postings[i]=[len(set(postings[i])),list(set(postings[i]))]
  dictionary_items = postings.items()
  for i in dictionary_items:
    print(i)
  #print(postings)

#term->[frequency,[position]]
pos_index = generate_positional_index(final_filtered_Sentence)
pos_index

('geosolut', [2, [0, 412]])
('technolog', [116, [0, 3079, 4615, 3085, 4110, 4116, 2072, 5155, 5162, 4140, 558, 5167, 4145, 4149, 59, 1597, 4158, 1600, 5191, 5709, 1105, 2129, 1629, 1124, 3685, 1638, 109, 3181, 2675, 2166, 2176, 2696, 137, 657, 5780, 5271, 1689, 672, 4256, 3237, 3238, 679, 2216, 5288, 682, 3242, 3248, 1719, 5816, 1724, 1215, 2752, 712, 3272, 4815, 720, 5329, 2263, 4823, 5342, 3308, 757, 758, 4341, 4864, 4869, 2828, 5389, 3353, 5410, 300, 1329, 2357, 823, 849, 1879, 4439, 4449, 2936, 5496, 5497, 4481, 1925, 4497, 1941, 3993, 412, 3484, 5534, 3487, 4515, 427, 941, 1970, 436, 2484, 1462, 2487, 3510, 5566, 4048, 3031, 473, 1499, 5084, 990, 4578, 5604, 2534, 1516, 3564, 3056, 4082, 1018, 3067, 4093]])
('leverag', [3, [0, 858, 3958]])
('benefon', [9, [0, 1312, 3808, 4993, 5604, 300, 4158, 1053, 3198]])
('s', [990, [0, 2, 2051, 2053, 6, 2055, 8, 9, 4102, 2059, 4107, 2061, 14, 17, 18, 2066, 23, 4124, 30, 31, 2079, 4126, 2082, 2083, 2084, 4127, 2086, 4128, 2088, 41, 4135, 2091, 

('nq', [1, [196]])
('hard', [6, [4161, 5539, 196, 2673, 1298, 5778]])
('found', [20, [2692, 4741, 2439, 2082, 1955, 1701, 1070, 2358, 5306, 196, 1875, 859, 3805, 4327, 2920, 747, 5748, 5494, 4728, 889]])
('support', [69, [1024, 5249, 258, 1410, 3203, 517, 774, 4999, 649, 1931, 5517, 1682, 787, 3860, 5396, 413, 2718, 1568, 546, 930, 2981, 2344, 2472, 938, 3753, 1713, 5810, 5427, 2357, 2487, 1976, 4027, 1724, 4412, 4413, 5310, 196, 453, 837, 2886, 712, 1096, 3527, 2895, 4815, 4049, 4177, 1749, 1750, 4567, 3800, 4824, 5589, 5590, 2783, 4961, 739, 3300, 3555, 358, 3046, 1896, 4584, 4967, 1646, 5234, 4851, 373, 2813]])
('stkscosz', [1, [196]])
('avc', [2, [197, 4709]])
('systemhau', [2, [197, 4709]])
('establish', [35, [4481, 4993, 267, 3602, 5018, 1949, 4383, 1188, 2853, 2601, 1974, 4409, 2753, 4034, 2243, 3524, 5313, 198, 840, 5705, 1612, 593, 2005, 1750, 1878, 4443, 5083, 355, 3432, 4329, 2671, 3575, 3576, 1785, 4219]])
('station', [11, [4580, 3013, 198, 5609, 5483, 2735, 5469, 1178, 306

('bring', [24, [5380, 2695, 2827, 782, 3609, 5792, 4513, 939, 2476, 1073, 3378, 3654, 4423, 3785, 1482, 3658, 3916, 1774, 5744, 756, 761, 5245, 5758, 2559]])
('sh', [2, [5778, 757]])
('distributor', [7, [776, 4682, 5131, 844, 757, 5180, 4445]])
('konecran', [20, [4782, 4400, 5171, 5561, 5697, 3653, 4940, 1871, 4561, 5841, 853, 3671, 2012, 2144, 3041, 2665, 4074, 2927, 2035, 757]])
('fiveyear', [5, [3900, 757, 4380, 824, 2168]])
('fuel', [10, [1984, 4900, 2055, 757, 1462, 2357, 2485, 2613, 2203, 3132]])
('meter', [6, [3010, 3984, 2962, 758, 792, 1215]])
('spend', [5, [1472, 3009, 2754, 5481, 759]])
('proce', [3, [5271, 1455, 759]])
('sheet', [10, [1701, 3846, 1863, 2824, 4059, 2580, 1910, 759, 1691, 4287]])
('rxii', [1, [761]])
('tang', [1, [761]])
('realli', [5, [3176, 5741, 1682, 761, 3675]])
('toxic', [3, [761, 3805, 4741]])
('least', [10, [960, 4868, 2822, 3882, 3886, 4252, 3443, 761, 985, 1535]])
('drag', [5, [1831, 817, 3763, 1428, 761]])
('consortium', [3, [3601, 762, 4957]])
('o

('fallen', [5, [1447, 4396, 1682, 4279, 5305]])
('wwwruukkicominvestor', [1, [1448]])
('hayward', [1, [1450]])
('califbas', [1, [1450]])
('casual', [1, [1450]])
('footwear', [1, [1450]])
('boot', [2, [3369, 1450]])
('sandal', [1, [1450]])
('rostelecom', [2, [1452, 3206]])
('rtkm', [1, [1452]])
('transmiss', [3, [1795, 1454, 2415]])
('applianc', [1, [1454]])
('modul', [6, [1454, 2449, 5138, 3544, 4793, 5757]])
('chroark', [1, [1456]])
('unxl', [1, [1456]])
('ascend', [4, [1456, 2401, 2906, 3649]])
('stkscoatglook', [1, [1456]])
('fujitsu', [3, [1458, 3661, 3398]])
('ntt', [2, [1458, 2132]])
('docomo', [1, [1458]])
('juergen', [1, [1458]])
('walter', [3, [1458, 1990, 5343]])
('depositari', [2, [4336, 1459]])
('pledg', [2, [2538, 1461]])
('render', [2, [1805, 1461]])
('desktop', [3, [2784, 1461, 3741]])
('rori', [1, [1462]])
('fitzgerald', [1, [1462]])
('bristol', [2, [2144, 1462]])
('port', [16, [2144, 4640, 2021, 5318, 1865, 4971, 2157, 2287, 1776, 1906, 1813, 1462, 5527, 5593, 4698, 18

('bawag', [1, [2565]])
('altia', [2, [5338, 2570]])
('vuokatti', [1, [2572]])
('prototyp', [1, [2572]])
('rampup', [1, [2572]])
('noncoher', [1, [2573]])
('matrix', [1, [2573]])
('abovement', [1, [2582]])
('kenget', [1, [2585]])
('captur', [3, [5690, 4670, 2591]])
('nevski', [3, [2592, 5609, 4086]])
('admit', [2, [4777, 2594]])
('penalti', [1, [2594]])
('ceivd', [1, [2596]])
('warranti', [1, [2596]])
('blyk', [1, [2599]])
('petrofac', [2, [2600, 4592]])
('shetland', [1, [2600]])
('borl+ñng', [2, [2601, 4868]])
('marchapril', [1, [2601]])
('ukbas', [1, [2602]])
('cogener', [1, [2603]])
('secondbiggest', [1, [2603]])
('kauna', [2, [5218, 2603]])
('dlerch', [1, [2604]])
('fmcn', [1, [2604]])
('darn', [1, [2604]])
('chit', [1, [2604]])
('sherlock', [1, [2604]])
('inci', [1, [2606]])
('relinquish', [1, [2608]])
('lappeenranta', [1, [2611]])
('feedstock', [1, [2613]])
('palm', [4, [3705, 3341, 3027, 2613]])
('hover', [1, [2614]])
('dialog', [1, [2614]])
('mwe', [1, [2619]])
('toast', [1, [26

('nonperform', [1, [4078]])
('businesstobusi', [1, [4080]])
('netel', [2, [4081, 4196]])
('cheaper', [2, [4081, 4196]])
('seventythre', [1, [4082]])
('archestra', [1, [4082]])
('wonderwar', [2, [4082, 4999]])
('contempu', [1, [4083]])
('child', [1, [4087]])
('budapest', [3, [4096, 5153, 4142]])
('egeszsegbolt', [1, [4096]])
('onti', [1, [4101]])
('debat', [1, [4101]])
('linger', [1, [4101]])
('ont', [1, [4101]])
('krav', [2, [4262, 4102]])
('facilit', [1, [4102]])
('traceyryniec', [1, [4107]])
('wwg', [1, [4107]])
('kaman', [1, [4107]])
('ddd', [1, [4109]])
('jefferi', [1, [4109]])
('stkscogcqd', [1, [4109]])
('mou', [1, [4110]])
('aegn', [1, [4113]])
('adx', [1, [4113]])
('stkscoqzzx', [1, [4113]])
('sidelin', [1, [4115]])
('uv', [1, [4116]])
('wastewat', [2, [4297, 4116]])
('trojan', [2, [4116, 4670]])
('logic', [1, [4116]])
('ballast', [1, [4116]])
('kaluga', [1, [4118]])
('geneva', [1, [4119]])
('tero', [1, [4119]])
('aaltonen', [1, [4119]])
('ticaret', [1, [4122]])
('sanayi', [1, 

In [17]:
text = df['Sentence'].head(100)
distinct_words = []
#Positional Posting List
for words in text:
    distinct_words.extend(words.split(' '))

tokens = list(set(distinct_words))
len(tokens)

#Positional Index
positional_posting_list = {}
for word in tokens:
    lst = []
    positional_posting_list[word] = [distinct_words.count(word)]
    for i in range(0, len(text)):
        j = text[i].split()
        dic = {} 
        dic[i] = []
        if (word in j):
            list_words = j
            dic[i].extend([word_pos+1 for word_pos in range(len(list_words)) if list_words[word_pos]==word ])
            lst.append(dic)
    positional_posting_list[word].extend(lst)
        
positional_posting_list

{'': [1],
 'Inc.': [1, {67: [5]}],
 'fade': [1, {63: [8]}],
 '`': [4, {36: [7]}, {85: [1]}, {95: [5, 14]}],
 '256.88': [1, {50: [21]}],
 '14.3': [1, {75: [13]}],
 '$aapl': [1, {92: [1]}],
 "Shell's": [1, {6: [1]}],
 'floral': [1, {41: [17]}],
 'all': [3, {3: [9]}, {30: [35]}, {56: [26]}],
 'Nasdaq': [1, {19: [7]}],
 'lose': [1, {39: [12]}],
 'is': [11,
  {18: [5]},
  {19: [3]},
  {22: [12, 19]},
  {31: [20]},
  {35: [3]},
  {46: [12]},
  {54: [4]},
  {85: [3]},
  {86: [5]},
  {92: [3]}],
 'May.': [1, {60: [7]}],
 'Development': [1, {62: [15]}],
 'attract': [1, {72: [6]}],
 'm': [3, {37: [9]}, {71: [12]}, {89: [21]}],
 '1,500': [1, {82: [12]}],
 'margin': [1, {84: [14]}],
 '13.32': [1, {31: [9]}],
 'more:': [1, {65: [19]}],
 'earlier': [5, {2: [23]}, {17: [22]}, {37: [14]}, {73: [41]}, {83: [27]}],
 'talking': [1, {25: [10]}],
 'GS': [1, {93: [1]}],
 'topical': [1, {96: [6]}],
 'USD': [3, {46: [15]}, {47: [19]}, {73: [21]}],
 '0.50': [1, {99: [19]}],
 'Kauko-Telko': [1, {70: [1]}],
 'pr

In [13]:
#simple phrase/word query
import time
def get_word_postings(word):
    flag = False
    start=time.time()
    dictionary_items = postings.items()
    for i in dictionary_items:
        if(i[0] == word):
            flag = True
            print(i)
            break
        else:
            time.sleep(0.0000000001)
            continue
    end=time.time()
    time_taken=end-start       #Time
    if flag:
        print("Time taken to fetch (simple phrase/word query): ",time_taken,"seconds")
    else:
        print("Could not find the word:",word)    

In [14]:
get_word_postings("low")

('low', [39, [3200, 1, 769, 1029, 1552, 2449, 2066, 5138, 3863, 3737, 416, 4897, 5667, 5797, 550, 1961, 173, 1843, 2357, 310, 1462, 2614, 1465, 1602, 1107, 3414, 600, 3289, 4572, 2398, 5093, 3303, 3946, 5227, 4079, 757, 2425, 382, 383]])
Time taken to fetch (simple phrase/word query):  0.34474611282348633 seconds


In [15]:
filtered_Sentence = df['Final'] 

In [16]:
#boolean query (Intersection)
def get_intersection_postings(word1, word2):
    flag = False
    start=time.time()
    #locating words in postings dictionary
    required = []
    answer = {}
    dictionary_items = postings.items()
    for i in dictionary_items:
        if(i[0] == word1):
            required.append(i)
        if(i[0] == word2):
            required.append(i)
        else:
            continue
            
    #print(required)
    
    indexes = []
    list1 = []
    list2 = []
    
    #Finding the intersection
    for i in required:
        #print(i)
        word, posting2 = i
        #print(posting2)
        frequency, index = posting2[0], posting2[1]
        #print(index)
        indexes.append(index)
        #print(indexes)

    list1, list2 = indexes[0], indexes[1]
      
    #print(list1)
    #print(list2)
    list3 = [value for value in list1 if value in list2]
    #print(list3)
    #answer[word1+ " AND " + word2]=[len(set(list3)),list(set(list3))]
    answer[word1+ " AND " + word2]= list(set(list3))
    
    end=time.time()
    time_taken=end-start       #Time
    
    if len(list3):
        print(answer)
        print('\n\n')
        print("Time taken to fetch (boolean query Intersection): ",time_taken,"seconds")
        print('\n\n')
    else:
        print('\n\n')
        print("No intersection possible")
        print('\n\n')

In [17]:
get_intersection_postings("esi","low")

{'esi AND low': [1]}



Time taken to fetch (boolean query Intersection):  0.0 seconds





In [18]:
get_intersection_postings("helsinki","afx")

{'helsinki AND afx': [4515, 3653, 3238, 2757, 1773, 494, 208, 5841, 4790, 1718, 4891, 1052]}



Time taken to fetch (boolean query Intersection):  0.007993459701538086 seconds





In [20]:
print("Enter query")
query = input()
for i in query.split():
    get_word_postings(i)

Enter query
esi low helsinki afx
('esi', [1, [1]])
Time taken to fetch (simple phrase/word query):  0.36565423011779785 seconds
('low', [39, [3200, 1, 769, 1029, 1552, 2449, 2066, 5138, 3863, 3737, 416, 4897, 5667, 5797, 550, 1961, 173, 1843, 2357, 310, 1462, 2614, 1465, 1602, 1107, 3414, 600, 3289, 4572, 2398, 5093, 3303, 3946, 5227, 4079, 757, 2425, 382, 383]])
Time taken to fetch (simple phrase/word query):  0.33396100997924805 seconds
('helsinki', [153, [1536, 1026, 2050, 5125, 4614, 2059, 18, 19, 531, 2578, 4630, 2072, 5658, 3611, 1052, 3104, 1057, 3626, 4141, 1586, 3638, 1600, 1089, 2117, 3141, 3653, 2120, 5701, 4686, 593, 1105, 3155, 2134, 5206, 3162, 5723, 609, 610, 612, 101, 616, 5736, 4714, 3583, 2672, 3185, 5236, 1655, 4736, 1665, 2690, 4605, 132, 3205, 5761, 5766, 137, 5770, 5792, 3237, 3238, 2227, 3253, 1718, 183, 1719, 1209, 2234, 4790, 5313, 4291, 2757, 4805, 5318, 5831, 4297, 208, 721, 1233, 5841, 1238, 3297, 1772, 1773, 254, 4872, 3850, 2318, 3854, 2327, 3864, 4891, 79

In [21]:
print("Enter 2 words for boolean query processing (Intersection)")
word1 = input()
word2 = input()
get_intersection_postings(word1,word2)

Enter 2 words for boolean query processing (Intersection)
helsinki
afx
{'helsinki AND afx': [4515, 3653, 3238, 2757, 1773, 494, 208, 5841, 4790, 1718, 4891, 1052]}



Time taken to fetch (boolean query Intersection):  0.006004810333251953 seconds





In [22]:
#boolean query (Union)
def get_union_postings(word1, word2):
    flag = False
    start=time.time()
    #locating words in postings dictionary
    required = []
    answer = {}
    dictionary_items = postings.items()
    for i in dictionary_items:
      if(i[0] == word1):
          required.append(i)
      if(i[0] == word2):
          required.append(i)
      else:
        continue
    #print(required)

    indexes = []
    list1 = []
    list2 = []
    #Finding the union
    for i in required:
      #print(i)
      word, posting2 = i
      #print(posting2)
      frequency, index = posting2[0], posting2[1]
      #print(index)
      indexes.append(index)
    #print(indexes)

    list1, list2 = indexes[0], indexes[1]
      
    #print(list1)
    #print(list2)
    list3 = list1 + list2 
    #print(list3)
    #answer[word1+ " OR " + word2]=[len(set(list3)),list(set(list3))]
    answer[word1+ " OR " + word2]= list(set(list3))
    end=time.time()
    time_taken=end-start       #Time
    if len(list3):
      print(answer)
      print("Time taken to fetch (boolean query Union): ",time_taken,"seconds")
    else:
      print("No Union possible")

In [23]:
print("Enter 2 words for boolean query processing (Union)")
word1 = input()
word2 = input()
get_union_postings(word1,word2)

Enter 2 words for boolean query processing (Union)
esi
helsinki
{'esi OR helsinki': [1536, 1, 1026, 2050, 509, 5125, 4614, 2059, 18, 19, 531, 2578, 4630, 2072, 5658, 3611, 1052, 3104, 1057, 3626, 4141, 1586, 3638, 2047, 1600, 1089, 2117, 3141, 3653, 2120, 5701, 4686, 593, 1105, 3155, 2134, 5206, 3162, 5723, 609, 610, 612, 101, 616, 5736, 4714, 2672, 3185, 5236, 1655, 4736, 1665, 2690, 5761, 132, 3205, 5766, 137, 5770, 5792, 3237, 3238, 2227, 3253, 1718, 183, 1719, 1209, 2234, 4790, 5313, 4291, 2757, 4805, 5318, 5831, 4297, 208, 721, 1233, 5841, 1238, 3297, 1772, 1773, 254, 4872, 3850, 2318, 3854, 2327, 3864, 4891, 796, 4386, 2342, 808, 1320, 3368, 2347, 1324, 815, 2871, 3895, 3390, 1344, 1859, 3907, 1864, 335, 4435, 3929, 4967, 2418, 2934, 3960, 892, 1405, 1920, 904, 5006, 4497, 5009, 2452, 5525, 3479, 409, 1434, 1947, 415, 4515, 1444, 3496, 427, 1965, 2482, 2488, 4538, 2500, 3018, 3020, 3022, 3033, 1499, 997, 4070, 5095, 494, 1010, 5108, 502, 4605, 4094, 3583]}
Time taken to fetch (bo

In [24]:
#For finding if two words occur together and in which document.
filtered_Sentence = df['Final'] 
def get_phrase_query(phrase): 
    
    start=time.time()
    str_to_process = phrase.split()

    i=0
    j=0

    lim1=0
    lim2=0

    ans=[]
    if (str_to_process[0] in postings) and (str_to_process[1] in postings):
        while (lim1<len(postings[str_to_process[0]][1]) and lim2<len(postings[str_to_process[1]][1]) ):
            if(postings[str_to_process[0]][1][i] == postings[str_to_process[1]][1][j]):
                ans.append(postings[str_to_process[1]][1][j])
                i+=1
                j+=1

            elif (postings[str_to_process[0]][1][i] < postings[str_to_process[1]][1][j]):
                i+=1

            else:
                j+=1

            lim1+=1
            lim2+=1

    else:
        print("Not found in any tweet")

    final_tweets_id=[]
    pos_idx = []
    for p in ans:
        held_for_now=filtered_Sentence[p].split()

        if( held_for_now.index(str_to_process[0]) == (held_for_now.index(str_to_process[1])-1) ):
            final_tweets_id.append(p)
            pos_idx.append(len(final_tweets_id))
            pos_idx.append(final_tweets_id)
    
    end=time.time()
    time_taken=end-start 

    print("The phrase is present in tweet ids:",pos_idx)
    print("Time taken to fetch the phrase query: ",time_taken,"seconds")

In [25]:
get_phrase_query("geosolut technolog")

The phrase is present in tweet ids: [1, [0]]
Time taken to fetch the phrase query:  0.0 seconds
